# lib import


In [1]:
from datetime import date
from fyers_api import fyersModel
from fyers_api import accessToken
import os
from dotenv import load_dotenv
from fyers_api.Websocket import ws
import datetime as dt
load_dotenv()

False

# details

In [2]:


# Replace these values with your actual API credentials
client_id = "4XTKPAEOV2-100"
secret_key = "ZZOS6DOV1V"
redirect_uri = "https://fyers.in/"

today = date.today().strftime("%Y-%m-%d")


def get_access_token():
    access = ""
    if not os.path.exists("./authcode"):
        print("Creating authcode directory")
        os.makedirs("./authcode")

    if not os.path.exists(f"authcode/{today}.txt"):
        session = accessToken.SessionModel(client_id=client_id, secret_key=secret_key,
                                           redirect_uri=redirect_uri, response_type="code", grant_type="authorization_code")
        response = session.generate_authcode()
        print("Login Url : ", response)
        auth_code = input("Enter Auth Code : ")
        session.set_token(auth_code)
        access_token = session.generate_token()["access_token"]
        with open(f"authcode/{today}.txt", "w") as f:
            f.write(access_token)
    else:
        with open(f"authcode/{today}.txt", "r") as f:
            access = f.read()
    return access
access_token = get_access_token()

# for the profile

In [3]:
fyers = fyersModel.FyersModel(
    client_id=client_id, token=access_token, log_path=os.getcwd())

print(fyers.get_profile())
print(fyers.funds())
print(fyers.holdings())


{'s': 'ok', 'code': 200, 'message': '', 'data': {'fy_id': 'XS99371', 'name': 'SANTHOSH KUMAR KOMMIRISHETTY', 'image': None, 'display_name': None, 'pin_change_date': '21-03-2024 22:15:23', 'email_id': 'santhoshkumark.komirishetty@gmail.com', 'pwd_change_date': None, 'PAN': '---------', 'mobile_number': '9963381522', 'totp': True, 'pwd_to_expire': 90}}
{'s': 'ok', 'code': 200, 'message': '', 'fund_limit': [{'id': 3, 'title': 'Clear Balance', 'equityAmount': 0, 'commodityAmount': 0}, {'id': 1, 'title': 'Total Balance', 'equityAmount': 0, 'commodityAmount': 0}, {'id': 7, 'title': 'Receivables', 'equityAmount': 0, 'commodityAmount': 0}, {'id': 4, 'title': 'Realized Profit and Loss', 'equityAmount': 0, 'commodityAmount': 0}, {'id': 2, 'title': 'Utilized Amount', 'equityAmount': 0, 'commodityAmount': 0}, {'id': 5, 'title': 'Collaterals', 'equityAmount': 0, 'commodityAmount': 0}, {'id': 8, 'title': 'Adhoc Limit', 'equityAmount': 0, 'commodityAmount': 0}, {'id': 10, 'title': 'Available Balance'

# for the history data

In [4]:
 data = {"symbol": "NSE:NIFTYBANK-INDEX", "resolution": "5", "date_format": "1",
         "range_from": "2023-01-18", "range_to": "2023-01-18", "cont_flag": "1"}
data ={"symbol":"NSE:PIIND-EQ","resolution":"D","date_format":"1","range_from":"2024-07-18","range_to":"2024-07-18","cont_flag":"1"}
 print(fyers.history(data))

{'s': 'ok', 'candles': [[1721260800, 3894.0, 3928.3, 3849.05, 3905.0, 330915]]}


In [5]:
data = {"symbol":"NSE:SBIN-EQ","resolution":"D","date_format":"0","range_from":"1622097600","range_to":"1622097685","cont_flag":"1"}

print(fyers.history(data))
print("---------------------------------------")
data = {"symbols":"NSE:SBIN-EQ"}
print(fyers.quotes(data))

{'s': 'ok', 'candles': [[1622073600, 413.7, 429.1, 412.0, 425.2, 73392997]]}
---------------------------------------
{'message': '', 'code': 200, 'd': [{'n': 'NSE:SBIN-EQ', 'v': {'ask': 896.15, 'bid': 895.85, 'chp': 0.26, 'ch': 2.3, 'description': 'NSE:SBIN-EQ', 'exchange': 'NSE', 'fyToken': '10100000003045', 'high_price': 898.8, 'low_price': 883.3, 'lp': 895.85, 'open_price': 895, 'original_name': 'NSE:SBIN-EQ', 'prev_close_price': 893.55, 'short_name': 'SBIN-EQ', 'spread': 0.3, 'symbol': 'NSE:SBIN-EQ', 'tt': '1721347200', 'volume': 9116384}, 's': 'ok'}], 's': 'ok'}


In [6]:
print(fyers.convert_position({"symbol":"MCX:SILVERMIC20AUGFUT","positionSide":"1","convertQty":"1","convertFrom":"MARGIN","convertTo":"INTRADAY"}))


{'s': 'error', 'code': '-310', 'message': 'Please provide a valid symbol'}


# for the live tick data 

In [ ]:
newToken = f"{client_id}:{access_token}"
symbol = ["NSE:NIFTYBANK-INDEX"]
cws = ws.FyersSocket(access_token=newToken,
                     run_background=False, log_path=os.getcwd())

def on_ticks(msg):
    #print(msg.length)
    print(msg)
    script = msg[0]['symbol']
    ltp = msg[0]['ltp']
    high = msg[0]['high_price']
    low = msg[0]['low_price']
    ltt = dt.datetime.fromtimestamp(msg[0]['timestamp'])
    print(
        f"Script:{script}  ,  ltp:{ltp}  ,  HIGH:{high} ,  LOW:{low},  ltt:{ltt}")


cws.websocket_data = on_ticks
cws.subscribe(symbol=symbol, data_type="symbolData")
cws.keep_running()
cws.unsubscribe(symbol=symbol)


[{'symbol': 'NSE:NIFTYBANK-INDEX', 'timestamp': 1721368521, 'fyCode': 7207, 'fyFlag': 2, 'pktLen': 48, 'ltp': 52338.4, 'open_price': 52531.55, 'high_price': 52586.75, 'low_price': 52146.3, 'close_price': 52620.7, 'min_open_price': 52337.4, 'min_high_price': 52346.3, 'min_low_price': 52335.75, 'min_close_price': 52338.4, 'min_volume': 0}]
Script:NSE:NIFTYBANK-INDEX  ,  ltp:52338.4  ,  HIGH:52586.75 ,  LOW:52146.3,  ltt:2024-07-19 11:25:21
[{'symbol': 'NSE:NIFTYBANK-INDEX', 'timestamp': 1721368522, 'fyCode': 7207, 'fyFlag': 2, 'pktLen': 48, 'ltp': 52345.15, 'open_price': 52531.55, 'high_price': 52586.75, 'low_price': 52146.3, 'close_price': 52620.7, 'min_open_price': 52337.4, 'min_high_price': 52346.3, 'min_low_price': 52335.75, 'min_close_price': 52345.15, 'min_volume': 0}]
Script:NSE:NIFTYBANK-INDEX  ,  ltp:52345.15  ,  HIGH:52586.75 ,  LOW:52146.3,  ltt:2024-07-19 11:25:22
[{'symbol': 'NSE:NIFTYBANK-INDEX', 'timestamp': 1721368524, 'fyCode': 7207, 'fyFlag': 2, 'pktLen': 48, 'ltp': 523

In [ ]:
fyers.place(